# Hedge Indicator
### Building hedge indicator dataframe from sec-api
Step 1: create df with all positive matches  
Step 2: create df with all false positive matches  
Step 3: create df with all file listings  
Step 4: merge  

In [1]:
import pandas as pd
import pyreadstat
import numpy as np

In [2]:
##Load in gvkey List
gv_list, meta = pyreadstat.read_dta('gvkey_list.dta')
gv_list.head()

,gvkey
0,210835
1,210418
2,029751
3,028349
4,001331


In [3]:
## Sort list & change to int
gv_list = gv_list.sort_values(by = 'gvkey')
gv_list = gv_list.drop(19)
gv_list['gvkey'] = gv_list['gvkey'].astype(str).astype(int)
gv_list.head()

,gvkey
3161,1004
2210,1013
3568,1019
1857,1021
21557,1025


In [4]:
##Load in reference map
cikmap = pd.read_csv('index.csv')
cikmap.set_index(keys = cikmap['gvkey'], inplace=True) ##gvkeys as indices
cikmap['cik']=cikmap['cik'].fillna(-1)
cikmap.head(10)

,gvkey,conm,DATADATE1,DATADATE2,cik,SOURCE,coname,FNDATE,LNDATE,N10K,N10K_NT,N10K_A,N10Q,N10Q_NT,N10Q_A,NDEF,N8K,NTOT,FLAG
gvkey,,,,,,,,,,,,,,,,,,,
1000,1000,A & E PLASTIK PAK INC,31DEC1964,31DEC1977,-1.0,NOLNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1001,1001,A & M FOOD SERVICES INC,31DEC1978,31DEC1985,723576.0,COMPN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1002,1002,AAI CORP,31DEC1968,31DEC1972,1745.0,COMPH,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1002,1002,AAI CORP,31DEC1968,31DEC1972,1306124.0,COMPN,AAI CORP,12NOV2004,11MAY2005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,3
1003,1003,A.A. IMPORTING CO INC,31DEC1980,31JAN1990,730052.0,COMPN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1004,1004,AAR CORP,31MAY1966,31MAY2022,1750.0,COMPN,AAR CORP,13JAN1994,01SEP2022,29.0,0.0,0.0,88.0,0.0,2.0,36.0,240.0,1976.0,3
1005,1005,A.B.A. INDUSTRIES INC,31OCT1974,31OCT1981,-1.0,NOLNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1006,1006,ABC INDS INC,30JUN1974,30JUN1982,-1.0,NOLNK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1007,1007,ABKCO INDUSTRIES INC,30SEP1974,30SEP1984,1882.0,COMPN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Positive Matches

In [8]:
positive_df = pd.DataFrame(columns=['CIK','Company_Name','Filing','Date','Hedge_Positive', 'Hedge_FalsePositive'])
positive_df.head()

,CIK,Company_Name,Filing,Date,Hedge_Positive,Hedge_FalsePositive


In [9]:
##import packages
from sec_api import FullTextSearchApi
queryApi = FullTextSearchApi(api_key="86712fa19f2b64be72ace0aa5aef5c749db89848915282aeaff562be7fb018fb")

##for each entry in gv_list
for index in range(5):
    
    ##Find first company CCIK
    key = gv_list['gvkey'].iloc[index]
    cik = cikmap['cik'].loc[key].astype(int).astype(str)
    print("CIK for Company " + str(index) + ": " + str(cik))
    
    ##check if CIK is in list
    if (cik != '-1'):
        ##Download
        query = {
            "query": "\"interest rate swap\" OR \
                \"hedging against interest rate\" OR \
                \"hedge interest rate\" OR \
                \"hedges interest rate\" OR \
                \"hedging interest rate\" OR \
                \"hedge for interest rate\" OR \
                \"hedges for interest rate\" OR \
                \"hedging for interest rate\" OR \
                \"hedging of interest rate\" OR \
                \"interest rate hedge\" OR \
                \"interest rate hedging\" OR \
                \"interest rate risk hedge\" OR \
                \"interest rate risk hedging\" OR \
                \"interest rate derivative\" OR \
                \"interest rate swap\" OR \
                \"interest rate contract\" OR \
                \"interest rate agreement\" OR \
                \"interest rate collar\" OR \
                \"interest rate cap\"",
            "formTypes": ["10-K","10-Q"],
            "ciks": [cik],
            "startDate": "1900-01-01",
            "endDate": "2022-09-18"
        }
        filings = queryApi.get_filings(query)
        total = filings['total']['value']
        print("Total for Company " + str(index) + ": " + str(total))
        
        ##check if it has matches
        if (total > 0):

            ##create entries for given firm
            entries = []
            for i in range(len(filings['filings'])):
                cik = filings['filings'][i]['cik']
                name = filings['filings'][i]['companyNameLong']
                form = filings['filings'][i]['formType']
                date = filings['filings'][i]['filedAt']
                hedge_p = 1

                entry = [cik,name,form,date,hedge_p]
                entries.append(entry)

            comp_df = pd.DataFrame(entries, columns=['CIK','Company_Name','Filing','Date','Hedge_Positive'])
            comp_df['Date'] = pd.to_datetime(comp_df['Date'])
            comp_df = comp_df.sort_values(by=['Date'])
            comp_df['Hedge_FalsePositive'] = ''

            ##concat to hedge_df
            positive_df = pd.concat([positive_df,comp_df])
        
        ##print size of positive_df
        print("Total Entries: " + str(len(positive_df)) + "\n")

CIK for Company 0: 1750
Total for Company 0: 23
Total Entries: 23

CIK for Company 1: 61478
Total for Company 1: 12
Total Entries: 35

CIK for Company 2: 2668
Total for Company 2: 0
Total Entries: 35

CIK for Company 3: 319126
Total for Company 3: 13
Total Entries: 48

CIK for Company 4: 2880
Total for Company 4: 0
Total Entries: 48



In [10]:
##cleaning up
positive_df = positive_df.reset_index().drop(columns=['index'])
positive_df.head(50)

,CIK,Company_Name,Filing,Date,Hedge_Positive,Hedge_FalsePositive
0,1750,AAR CORP (AIR) (CIK 0000001750),10-K,2002-08-26 00:00:00,1,
1,1750,AAR CORP (AIR) (CIK 0000001750),10-K,2002-08-26 00:00:00,1,
2,1750,AAR CORP (AIR) (CIK 0000001750),10-Q,2011-09-23 00:00:00,1,
3,1750,AAR CORP (AIR) (CIK 0000001750),10-Q,2011-12-22 00:00:00,1,
4,1750,AAR CORP (AIR) (CIK 0000001750),10-Q,2012-03-26 00:00:00,1,
5,1750,AAR CORP (AIR) (CIK 0000001750),10-K,2012-07-19 00:00:00,1,
6,1750,AAR CORP (AIR) (CIK 0000001750),10-Q,2012-09-25 00:00:00,1,
7,1750,AAR CORP (AIR) (CIK 0000001750),10-Q,2012-12-21 00:00:00,1,
8,1750,AAR CORP (AIR) (CIK 0000001750),10-Q,2013-03-22 00:00:00,1,
9,1750,AAR CORP (AIR) (CIK 0000001750),10-K,2013-07-26 00:00:00,1,


### False Positive Matches

In [32]:
f_positive_df = pd.DataFrame(columns=['CIK','Company_Name','Filing','Date','Hedge_FalsePositive', 'Hedge_Positive'])
f_positive_df.head()

,CIK,Company_Name,Filing,Date,Hedge_FalsePositive,Hedge_Positive


In [ ]:
with open("f_positive_clean.txt", "r") as keywords:
    search_string = keywords.read()
    print(search_string)

In [43]:
cikmap.loc[1045]
cikmap['cik'].loc[key].astype(int).astype(str)
cik = '6201'

In [46]:
#### import packages
from sec_api import FullTextSearchApi
queryApi = FullTextSearchApi(api_key="86712fa19f2b64be72ace0aa5aef5c749db89848915282aeaff562be7fb018fb")

# ##for each entry in gv_list
for index in range(1):
    ##Find first company CCIK
#     key = gv_list['gvkey'].iloc[index]
#     cik = cikmap['cik'].loc[key].astype(int).astype(str)
    print("CIK for Company " + str(index) + ": " + str(cik))
    
    ##check if CIK is in list
    if (cik != '-1'):
        ##Download
        query = {
            "query": "\"not used any interest rate\" OR \
                        \"not use any interest rate\" OR \
                        \"not using any interest rate\" OR \
                        \"not used interest rate\" OR \
                        \"not use interest rate\" OR \
                        \"not using interest rate\" OR \
                        \"not used an interest rate\" OR \
                        \"not use an interest rate\" OR \
                        \"not using an interest rate\" OR \
                        \"not currently use any interest rate\" OR \
                        \"not currently using any interest rate\" OR \
                        \"not currently use interest rate\" OR \
                        \"not currently using interest rate\" OR \
                        \"not currently use an interest rate\" OR \
                        \"not currently using an interest rate\" OR \
                        \"currently no interest rate\" OR \
                        \"not hedged against any interest rate\" OR \
                        \"not hedge against any interest rate\" OR \
                        \"not hedging against any interest rate\" OR \
                        \"not hedged against interest rate\" OR \
                        \"not hedge against interest rate\" OR \
                        \"not hedging against interest rate\" OR \
                        \"not hedged any interest rate\" OR \
                        \"not hedge any interest rate\" OR \
                        \"not hedging any interest rate\" OR \
                        \"not hedged interest rate\" OR \
                        \"not hedge interest rate\" OR \
                        \"not hedging interest rate\" OR \
                        \"not enter into any interest rate\" OR \
                        \"not enter into interest rate\" OR \
                        \"not enter into an interest rate\" OR \
                        \"not engaged in any interest rate\" OR \
                        \"not engage in any interest rate\" OR \
                        \"not engaged in interest rate\" OR \
                        \"not engage in interest rate\" OR \
                        \"not use derivative financial instruments as a hedge against interest rate\" OR \
                        \"termination of interest rate\" OR \
                        \"fixed to variable interest rate\" OR \
                        \"fixed to floating interest rate\" OR \
                        \"fixed rate debt to variable rate\" OR \
                        \"fixed rate debt to floating rate\" OR \
                        \"fixed to variable rate interest rate\" OR \
                        \"fixed to floating rate interest rate\" OR \
                        \"fixed rate to variable rate\" OR \
                        \"fixed rate to floating rate\" OR \
                        \"fixed rates to variable rates\" OR \
                        \"fixed rates to floating rates\" OR \
                        \"fixed rate into variable rate\" OR \
                        \"fixed rate into floating rate\" OR \
                        \"fixed rates into variable rates\" OR \
                        \"fixed rates into floating rates\" OR \
                        \"fixed rate obligations to variable rate obligations\" OR \
                        \"fixed rate obligations to floating rate obligations\" OR \
                        \"a fixed rate to a variable rate\" OR \
                        \"a fixed rate to a floating rate\" OR \
                        \"fixed to floating swap\" OR \
                        \"not engaged in hedge transactions such as interest rate futures contracts or interest rate swap\" OR \
                        \"not enter into derivative transactions or speculate on the future direction of interest rate\" OR \
                        \"not use derivative financial instruments such as interest rate swap\" OR \
                        \"not entered into any swap agreement\" OR \
                        \"not have any interest rate swap\" OR \
                        \"not have interest rate swap\" OR \
                        \"not have an interest rate swap\" OR \
                        \"had no interest rate swap\" OR \
                        \"not currently hedge or otherwise use derivative instruments to manage interest rate\" OR \
                        \"not attempt to reduce or eliminate our exposure to interest rate risk through the use of derivative financial instrument\" OR \
                        \"enter into variable interest rate swaps effectively converting fixed rate borrowings to variable rate\" OR \
                        \"not designate the interest rate swap as a cash flow hedge and the interest rate swap will not qualify for hedge accounting\" OR \
                        \"not participate in hedging programs interest rate swaps or other activities involving the use of derivative financial instruments to manage interest rate risk\" OR \
                        \"may use interest rate swaps to balance exposure to interest rate\" OR \
                        \"in the future we may enter into interest rate swap\" OR \
                        \"may also enter into derivative financial instruments such as interest rate swap\" OR \
                        \"may enter into interest rate hedging agreements in the future to mitigate our exposure to interest rate risk\" OR \
                        \"may enter into interest rate swaps involving the exchange of floating for fixed rate interest payments\" OR \
                        \"swap agreements to hedge a portion of the consolidated interest rate risk associated with issuances of fixed rate\" OR \
                        \"fixed rate receipts in exchange for making floating rate payments\" OR \
                        \"fixed rate amounts in exchange for making floating rate payments\" OR \
                        \"manage interest rate risk related to fixed rate borrowing\" OR \
                        \"not have any derivative instruments outstanding\" OR \
                        \"swaps changed the fixed rate exposure on the debt to variable\" OR \
                        \"terminated all of its interest rate swap agreements\" OR \
                        \"swap long term borrowings at fixed rates into variable rates\" OR \
                        \"we pay variable rates and we receive a fixed rate\" OR \
                        \"exchange an obligation to make fixed debt payments for an obligation to make floating rate payments',...\" OR \
                        \"were no interest rate swap agreements\" OR \
                        \"were no interest rate swaps outstanding\" OR \
                        \"receives fixed interest rate payments and makes variable interest rate payments\" OR \
                        \"receive fixed pay variable interest rate\" OR \
                        \"receive fixed pay floating interest rate\" OR \
                        \"swapped the fixed rate to a variable rate\" OR \
                        \"will receive fixed interest rate payments and will make variable interest rate payments\" OR \
                        \"not use derivative financial instruments to manage interest rate risk\" OR \
                        \"not currently use derivative financial instruments to manage interest rate risk\"",
            "formTypes": ["10-K","10-Q"],
            "ciks": [cik],
            "startDate": "1900-01-01",
            "endDate": "2022-09-18"
        }
        filings = queryApi.get_filings(query)
        total = filings['total']['value']
        print("Total for Company " + str(index) + ": " + str(total))
        
        ##check if it has matches
        if (total > 0):

            ##create entries for given firm
            entries = []
            for i in range(len(filings['filings'])):
                cik = filings['filings'][i]['cik']
                name = filings['filings'][i]['companyNameLong']
                form = filings['filings'][i]['formType']
                date = filings['filings'][i]['filedAt']
                hedge_f = 1

                entry = [cik,name,form,date,hedge_p]
                entries.append(entry)

            comp_df = pd.DataFrame(entries, columns=['CIK','Company_Name','Filing','Date','Hedge_FalsePositive'])
            comp_df['Date'] = pd.to_datetime(comp_df['Date'])
            comp_df = comp_df.sort_values(by=['Date'])
            comp_df['Hedge_Positive'] = ''
            
            ##concat to hedge_df
            f_positive_df = pd.concat([f_positive_df,comp_df])
        
        ##print size of positive_df
        print("Total Entries: " + str(len(f_positive_df)) + "\n")

CIK for Company 0: 6201


KeyError: 'value'